# NBA Player Data Analysis Project

## Project Outline

1. **Introduction**
   - Overview of the project
   - Goals and objectives

2. **Data Collection**
   - Import necessary libraries
   - Fetch player data from the NBA API

3. **Data Processing**
   - Load data into a Pandas DataFrame
   - Data cleaning and transformation

4. **Data Analysis**
   - Exploratory data analysis (EDA)
   - Visualizations

5. **Conclusion**
   - Summary of findings
   - Future work


## 0. Environment

### Virtual Environment

It's generally recommended that you use a virtual environment (or venv) for this project. That way, all dependencies can be installed for the project without affecting the rest of your system. You can create a venv with Python:

`python -m venv .venv`

To activate the virtual environment in your shell, you can use the following commands.

On Windows:

`.venv\Scripts\activate`

On other operating systems:

`.venv/bin/activate`

### Dependencies

This project uses [Poetry](https://python-poetry.org/) to manage its dependencies. You can install the dependencies with the `poetry` command:

`poetry install`

If you don't want to use Poetry, a `requirements.txt` is also provided. You can install this using `pip`:

`pip install -r requirements.txt`

### Imports

In [ ]:
import requests
import json
import time
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy

### Environment Variables

This project uses the [BALLDONTLIE](https://app.balldontlie.io/) API, which has a free key available [if you sign up for an account](https://app.balldontlie.io/signup). We will load the API key from the .env file if available.

In [ ]:
load_dotenv()
BALLDONTLIE_API_KEY = os.getenv("BALLDONTLIE_API_KEY")

### Notebook (Developer Only)

This notebook uses `nbstripout` to strip notebook output from Git commits. If you are committing code, please run the following command to set up the Git filter.

In [ ]:
!nbstripout --install

## 1. Data Collection

### Fetch Player Data from NBA API

Before we analyze the statistics for each players, we need to get a list of all players that had minutes in the 2023 season.

In [ ]:
BASE_URL = "https://api.balldontlie.io/v1/"
PLAYERS_ENDPOINT = "players"
PAGE_100 = "per_page=100"
HEADERS = {
    "Authorization": f"{BALLDONTLIE_API_KEY}"
}
ALL_PLAYERS_FILE = "../data/all_players.json"

if os.path.exists(ALL_PLAYERS_FILE):
    print(f"{ALL_PLAYERS_FILE} already exists.")
    with open(ALL_PLAYERS_FILE, "r") as f:
        all_players = json.load(f)
else:
    all_players = []
    next_cursor = None

    while True:
        if next_cursor:
            url = f"{BASE_URL}{PLAYERS_ENDPOINT}?{PAGE_100}&cursor={next_cursor}"
        else:
            url = f"{BASE_URL}{PLAYERS_ENDPOINT}?{PAGE_100}"
        
        response = requests.get(url, headers=HEADERS)
        
        if response.status_code == 200:
            data = response.json()
            all_players.extend(data["data"])
            next_cursor = data["meta"].get("next_cursor")
            
            if not next_cursor:
                break
            
            time.sleep(2)
        else:
            print(f"Request failed with status code {response.status_code}")
            break

    # Save all players data to a JSON file
    with open(ALL_PLAYERS_FILE, 'w') as f:
        json.dump(all_players, f, indent=4)

    print(f"All player data has been saved to {ALL_PLAYERS_FILE}")

print(all_players[0])
len(all_players)


This JSON file contains a list of all players in the NBA. We're only concerned with the players from the current season, so we need to eliminate all the players who aren't. One quick heuristic we can use is draft year. Obviously, a player who was drafted in 1986 will not be playing now. We want to pick a cutoff year as close as possible to the currrent year to eliminate as many players as we can. The easiest way to do this is check the oldest players still playing in the NBA, mark them as exceptions, and use their draft year as a starting point.
A list of the oldest players still in the NBA can be found at [this Wikipedia page](https://en.wikipedia.org/wiki/List_of_oldest_and_youngest_NBA_players#Active). We chose 2010 as the cutoff.

In [ ]:
OLD_PLAYERS = ["LeBron James", "Chris Paul", "Kyle Lowry", "PJ Tucker", "Kevin Durant", "Al Horford", "Mike Conley", "Jeff Green", "Derrick Rose", "Russell Westbrook", "Kevin Love", "Eric Gordon", "Brook Lopez", "Nicolas Batum", "DeAndre Jordan", "James Harden", "Stephen Curry", "DeMar DeRozan", "Jrue Holiday", "Taj Gibson", "Paul George"]
all_players_after_2010 = [player for player in all_players if player["draft_year"] == None or player["draft_year"] > 2010 or f"{player['first_name']} {player['last_name']}" in OLD_PLAYERS]
len(all_players_after_2010)

You'll notice that we included players that have `null` for their draft year. That's because those players are undrafted. There are some undrafted players currently in the NBA, so we can't exclude them purely based on that fact. The website [2KRatings](https://www.2kratings.com/) maintains [a list](https://www.2kratings.com/lists/undrafted-nba-players) of all active undrafted players. This includes players in the G League, however. We decided to take the top 35 players as, after that point, the players play so few minutes that their stats will have a negligible impact on analysis.

In [ ]:
UNDRAFTED_PLAYERS = ["Fred VanVleet", "Austin Reaves", "Naz Reid", "T.J. McConnell", "Luguentz Dort", "Alex Caruso", "Derrick Jones Jr.", "Duncan Robinson", "Simone Fontecchio", "Gary Payton II", "Max Strus", "Luke Kornet", "Jock Landale", "Christian Wood", "Caleb Martin", "Chris Boucher", "Dorian Finney-Smith", "Robert Covington", "Jose Alvarado", "Javonte Green", "Sam Hauser", "Keon Ellis", "Duop Reath", "Royce O'Neale", "Naji Marshall", "Scotty Pippen Jr.", "Haywood Highsmith", "Drew Eubanks", "Gabe Vincent", "Daniel Theis", "Maxi Kleber", "Jordan McLaughlin", "Jordan Goodwin", "Damion Lee", "Lamar Stevens"]
all_players_after_2010_without_undrafted = [player for player in all_players_after_2010 if player["draft_year"] != None or f"{player['first_name']} {player['last_name']}" in UNDRAFTED_PLAYERS]
len(all_players_after_2010_without_undrafted)

### Get Stats for each Player

Now that we've gotten our list of players, we can get the stats for each of them.

In [ ]:
STATS_ENDPOINT = "stats"
# we only need stats from the latest season
CURRENT_SEASON = "seasons[]=2023"
ALL_PLAYERS_STATS_FILE = "../data/all_players_stats.json"

if os.path.exists(ALL_PLAYERS_STATS_FILE):
    print(f"{ALL_PLAYERS_STATS_FILE} already exists.")
    with open(ALL_PLAYERS_STATS_FILE, "r") as f:
        all_players_stats = json.load(f)
else:
    all_players_stats = {}
    
    for index, player in enumerate(all_players_after_2010_without_undrafted):
        # we only need to query this once because there are fewer than 100 games in a season
        url = f"{BASE_URL}{STATS_ENDPOINT}?{PAGE_100}&{CURRENT_SEASON}&player_ids[]={player['id']}"
        response = requests.get(url, headers=HEADERS)

        if response.status_code == 200:
            data = response.json()
            all_players_stats.update({player["id"]: data["data"]})
            print(f"Downloaded {index} of {len(all_players_after_2010_without_undrafted)}", end="\r")
            time.sleep(2)
        else:
            print(f"Request failed with status code {response.status_code}")
            break
    
    with open(ALL_PLAYERS_STATS_FILE, "w") as f:
        json.dump(all_players_stats, f)
    
    print(f"All player data has been saved to {ALL_PLAYERS_STATS_FILE}")

print(all_players_stats["15"])

Although we've eliminated most of the players who aren't playing in the current season, some false positives remain. We can take care of the ones that didn't play at all by checking to see if their stats are an empty list.

In [ ]:
all_players_stats_current_season = {player_id: stats for player_id, stats in all_players_stats.items() if stats}
len(all_players_stats_current_season)

We're also only really concerned with players that have significant play time. We generally define this as having an average of at least 16 minutes per game.

In [ ]:
all_players_stats_current_season_with_16_mins = {player_id: stats for player_id, stats in all_players_stats_current_season.items() if sum(int(stat["min"]) for stat in stats) / len(stats) >= 16}
len(all_players_stats_current_season_with_16_mins)

Finally, we have to exclude games where the players didn't play as those would skew the results, i.e. games where they played 0 minutes.

In [ ]:
all_players_stats_current_season_with_16_mins_and_play_time = {player_id: [stat for stat in stats if int(stat["min"]) > 0] for player_id, stats in all_players_stats_current_season_with_16_mins.items()}

## Data Formatting

We used a Pandas dataframe to store these stats for each player. In addition to the direct statistics recorded like points and rebounds, we added an additional `is_home` indicator which is true when the game was played at home.

In [ ]:
df = pd.DataFrame()
for player_id, stats in all_players_stats_current_season_with_16_mins_and_play_time.items():
    desired_stats = ["fgm", "fga", "fg_pct", "fg3m", "fg3a", "fg3_pct", "ftm", "fta", "ft_pct", "oreb", "dreb", "reb", "ast", "stl", "blk", "turnover", "pf", "pts"]
    data = {
        "player_id": int(player_id),
        "player_name": f"{stats[0]['player']['first_name']} {stats[0]['player']['last_name']}",
    }
    data["min"] = [int(stat["min"]) for stat in stats]
    for desired_stat in desired_stats:
        data[desired_stat] = [stat[desired_stat] for stat in stats]
    data["is_home"] = [stat["game"]["home_team_id"] == stat["team"]["id"] for stat in stats]
    df_player = pd.DataFrame(data=data)
    df = pd.concat([df, df_player], ignore_index=True)
# df.to_excel("NBA_2023_Stats.xlsx")
df

Let's consolidate all of these 

In [ ]:
desired_stats = ["min", "fgm", "fga", "fg_pct", "fg3m", "fg3a", "fg3_pct", "ftm", "fta", "ft_pct", "oreb", "dreb", "reb", "ast", "stl", "blk", "turnover", "pf", "pts"]

df_grouped = df.groupby("player_id").agg(
    player_name=("player_name", "first"),
    # min=("min", "mean"),
    # fgm=("fgm", "mean"),
    # fga=("fga", "mean"),
    # fg_pct=("fg_pct", "mean"),
    fg3m=("fg3m", "mean"),
    fg3a=("fg3a", "mean"),
    fg3_pct=("fg3_pct", "mean"),
    # ftm=("ftm", "mean"),
    # fta=("fta", "mean"),
    # ft_pct=("ft_pct", "mean"),
    # oreb=("oreb", "mean"),
    # dreb=("dreb", "mean"),
    # reb=("reb", "std"),
    # ast=("ast", "mean"),
    # stl=("stl", "std"),
    # blk=("blk", "mean"),
    # turnover=("turnover", "mean"),
    # pf=("pf", "mean"),
    # pts=("pts", "var"),
    # is_home=("is_home", "mean")
)
# print(df.groupby("player_id")["pts"].std().reset_index().sort_values("pts"))
df_grouped_over_35 = df_grouped[df_grouped["fg3_pct"] > 0.35]
df_grouped_over_35_over_3 = df_grouped_over_35[df_grouped_over_35["fg3a"] > 3]
df_grouped_over_35_over_3.sort_values("fg3_pct")
df_grouped_over_35_over_3.to_excel("3 Point Stats for over 0.35 percentage and 3 makes average.xlsx")

In [ ]:
# distribution of shai's scores
import scipy.stats

stat = "pts"
df_jaylen = df.loc[df.loc[:, "player_id"] == 70, :]

plt.figure(figsize=(10, 6))
sns.histplot(df_jaylen.loc[:, stat], bins=20, kde=True, stat="density", color="skyblue", label="Steals Histogram")


mu, std = scipy.stats.norm.fit(df_jaylen.loc[:, stat])

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = scipy.stats.norm.pdf(x, mu, std)
plt.plot(x, p, "k", linewidth=2, label="Normal Distribution Fit")

# plt.xlabel("Number of Steals")
# plt.ylabel("Density")
# plt.title("Histogram of Steals with Normal Distribution Fit")

print(scipy.stats.jarque_bera(df_jaylen.loc[:, stat]))

plt.show()